<a href="https://colab.research.google.com/github/jyjoon001/gpt2-japanese/blob/master/visual_novel_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Visual Novel Generator

이 GPT-2 모델은 사전 학습된 일어 GPT-2 모델에 일본어 텍스트 어드벤처 게임에서 추출된 약 20만줄의 정제된 텍스트 데이터를 전이학습하여 Fine-Tuning하였습니다. 


Pre-Trained Model과 Training을 위한 기반 코드는 [Japanese GPT2 Generation Model](https://github.com/tanreinama/gpt2-japanese) 을 사용하였습니다. 
> [@jyjoon001](https://github.com/jyjoon001) , forked from [tanreinama.](https://github.com/tanreinama) 

In [3]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive


Google Drive에 파일이 있는지 확인하며 파일이 존재하지 않을 경우 Fine-Tune된 모델과 코드를 다운로드합니다. <br> 해당 파일은 1.1GB의 크기를 가지고 있습니다. 
* 적절한 웹호스팅 공간이 없어 임시로 Naver Mail 서버를 사용하고 있습니다.

In [12]:
import os
if os.path.isfile('/content/drive/MyDrive/visual-novel-generator/gpt2-generate.py') is True:
    print('path found.')
else:
    print('path not found, downloading model.')
    !wget https://bigfile.mail.naver.com/bigfileupload/download?fid=pXR5Wre8D6cZHqujKxgmFoMdax2mHqUmKoumFqg9KxbwFqEdHqurKo2maAvwaxvjF634KzFoMqg9FqUwp6EmF6trF4EZpxFoK6EdFAK9Mov=
    !unzip download?fid=pXR5Wre8D6cZHqujKxgmFoMdax2mHqUmKoumFqg9KxbwFqEdHqurKo2maAvwaxvjF634KzFoMqg9FqUwp6EmF6trF4EZpxFoK6EdFAK9Mov=

SyntaxError: ignored

starting_word를 조정해서 생성 모델이 어떤 후속 문장을 생성할지를 정할 수 있습니다.

In [10]:
starting_word = "けれど、あなたは私に夢と希望をくれました。" #하지만 당신은 저에게 꿈과 희망을 주었습니다.

필요한 코드를 실행시키기 위하여 필요한 라이브러리를 다운받습니다. 
</br> jaconv는 정제되지 않은 일어 데이터를 unicode 데이터로 normalize합니다.

In [11]:
%cd /content/drive/MyDrive/visual-novel-generator/
with open("prologue.txt", "w") as output:
     output.write(starting_word)
!pip install jaconv

[Errno 2] No such file or directory: '/content/drive/MyDrive/visual-novel-generator/'
/content/drive/MyDrive


gpt2-generate.py를 통하여 소스 코드를 실행시킵니다. 
해당 gpt2-generate.py는 다음과 같은 형태로 이루어져 있습니다. 
* argparse를 통하여 argument를 설정합니다. 사용하는 model, 입력/출력 문장, 최대/최소 음절과 문장이 얼마나 임의의 단어를 생성하는지에 대한 temperature 계수, top-k 및 top-p sampling 시 사용하는 어절의 갯수 등을 결정합니다. 
* 사용하는 Model의 정보를 불러온 뒤 layer 갯수 등의 정보를 결정합니다. 
* generate_one 함수에서는 우선 입력 문장을 encode하여  Tensorflow session을 실행시켜 후속 토큰을 생성하며, 이를 decode하여 자연어로 출력한다. 이 때```'<|endoftext|>'``` 토큰이 나올 경우는 생성된 문장에 last 라벨링을 하여 문장의 끝을 알 수 있게 합니다. 
* Tensorflow session의 경우 sampling.py의 sample_sequence를 통해서 output을 결정합니다. 이후, max_length까지 위의 함수를 통하여 문장을 generate합니다. 

In [ ]:
!python3 gpt2-generate.py --num_generate 1 --temperature 1.5 --max_length 1024 --min_length 512
from google.colab import files
files.download("script.txt")